`Time.now`是一个间接输入。首先我们引用了`Time`类，然后把`#now`消息发送给`Time`。我们想要的其实是方法`now`的返回值，而不是`Time`常量本身。任何时候为了得到方法返回值，而向其他对象发送消息，其实就是在使用间接输入。

间接输入层次越深，代码的耦合程度就越高

In [1]:
Place = Struct.new(:index, :name, :prize)

Place

In [2]:
first = Place.new(0, "first", "Peasant's Quest game")

#<struct Place index=0, name="first", prize="Peasant's Quest game">

In [3]:
second = Place.new(1, "second", "Limozeen Album")

#<struct Place index=1, name="second", prize="Limozeen Album">

In [4]:
thrid = Place.new(2, "third", "butter-da")

#<struct Place index=2, name="third", prize="butter-da">

In [5]:
winners = ["a", "b"]

["a", "b"]

In [6]:
[first, second, thrid].each do |place|
  p "In #{place.name} place, #{winners[place.index]}"
  p "you wind: #{place.prize}"
end

"In first place, a"
"you wind: Peasant's Quest game"
"In second place, b"
"you wind: Limozeen Album"
"In third place, "
"you wind: butter-da"


[#<struct Place index=0, name="first", prize="Peasant's Quest game">, #<struct Place index=1, name="second", prize="Limozeen Album">, #<struct Place index=2, name="third", prize="butter-da">]

`#to_s`就是一个显式转换方法。显式转换一般用于这样的情形：源类型和目标类型很大程度上不相关或者毫无关联

与之对应，`#to_str`就是一个隐式类型转换方法。隐式类型转换适用于元类型和目标类型很相近的情形

In [7]:
class EmacsConfigFile
  def initialize
    @filename = "#{ENV['HOME']}/.vimrc"
  end
  
  def to_path
    @filename
  end
end

:to_path

In [8]:
emacs_config = EmacsConfigFile.new

#<EmacsConfigFile:0x000000559842e170 @filename="/home/demouser/.vimrc">

In [9]:
File.open(emacs_config).lines.count

<main>: warning: IO#lines is deprecated; use #each_line instead


740

这是因为，`EmacsConfig`类定义了转换方法`#to_path`，而`File#open`又会在参数对象上调用`#to_path`方法，以便得到文件名字符串。这样一来，这个非字符串对象也能工作得很好。

下面的例子可以说明两者（隐式转化和显示转化的区别）。`Time`对象并非`String`，有太多方式可以将`Time`转化成`String`了。另外，这两种类型还是不想管的，因此`Time`只预定义了显式的类型转换方法`#to_s`，而没有定义隐式版的`#to_str`

In [10]:
now = Time.now
now.respond_to?(:to_s)
now.to_s
now.respond_to?(:to_str)

false

In [11]:
"hello, " + "world"

"hello, world"

如果`#to_str`所做的一切就是返回字符串自身，那么它看起来意义不大。`#to_str`的意义在于：许多`Ruby`方法都期望得到字符串输入，它们隐式地在输入对象上调用`#to_str`方法（这正是“隐式类型转换”这个术语的来源）

In [12]:
"I am a String".to_str

"I am a String"

In [13]:
class ArticleTitle
  def initialize(text)
    @text = text
  end
  
  def slug
    @text.strip.tr_s("A-Za-z0-0", "-").downcase
  end
  
  def to_str
    @text
  end
  
  def to_s
    to_str
  end
end

:to_s

In [14]:
title = ArticleTitle.new("A Modest Proposal")
"Today's future" +title

"Today's futureA Modest Proposal"

你可能还记得我说过Ruby核心类从来不会自动调用显式类型转换方法，但仍有例外，有一种情况下，Ruby语言却会自动调用显式类型转换方法。

In [15]:
"The time is now: #{Time.now}"

"The time is now: 2019-01-12 15:29:02 +0000"

In [16]:
PHONE_EXTENSIONS=["Operator", "Sales", "Customer Service"]

def dial_extension(dialed_number)
  dialed_number = dialed_number.to_i
  extension = PHONE_EXTENSIONS[dialed_number]
  puts "Please hold while you are connected to #{extension}"
end

dial_extension(nil)

Please hold while you are connected to Operator


如果只关心得到预期输入类型而不深究其源类型，则可以使用显式类型转化方法。

In [17]:
def set_centrifuge_speed(new_rpm)
  new_rpm = new_rpm.to_int
  puts "Adjusting centrifuge to #{new_rpm} RPM"
end

:set_centrifuge_speed

In [18]:
bad_input = nil
set_centrifuge_speed(bad_input)

NoMethodError: undefined method `to_int' for nil:NilClass

In [19]:
aaa = nil
"fe#{aaa}"

"fe"

某种情况下，如果方法的参数不符合预期类型或者与预期不接近，就会为程序带来隐患，这时最好用隐式类型转换

这里的隐式类型转换起到了“门卫”的作用：用以确保方法接受到的参数要么是`Integer`，要么可以很好地被转化为`Integer`。当参数不满足条件时，`NoMethodError`清楚地告诉了代码调用者调用非法。

In [20]:
def draw_line(start, endpoint)
  start = start.to_coords if start.respond_to?(:to_coords)
  start = start.to_ary
end

:draw_line

In [21]:
class Point
  attr_reader :x, :y, :name
  def initialize(x, y, name = nil)
    @x, @y, @name = x, y, name
  end
  
  def to_coords
    [x, y]
  end
end

:to_coords

In [22]:
start = Point.new(23,37)
endpoint = [45, 89]

draw_line(start, endpoint)

[23, 37]

方法就可以提供这样的接口：既可以接受普通输入，还能向前兼容那些更具语义的输入（如Point）

In [23]:
require 'forwardable'

class RecordCollection
  attr_accessor :records
  extend Forwardable
  def_delegator :@records, :[], :record_number
end

:record_number

In [24]:
r = RecordCollection.new
r.records = [4,5,6]
r.record_number(0)  # => 4

4

For example, say you have a class RecordCollection which contains an array @records. You could provide the lookup method record_number(), which simpley calls [] on the `records` array, like this

我们不能保证所有输入都是`Meter`类型而非Integer类型。我们也想支持那些能够用英尺表示，且能在运算中混合使用的度量单位。我们得想办法隐式地将`Meter`转换为`Feet`，以及将`Feet`转换为`Meter`

In [25]:
class Meters
  def to_meters
  end
end

class Feet
  def to_meters
  end
end

:to_meters

现在汇报还把变化，再也不用担心单位混淆了。因为我们确信任何不支持`to_meters`协议的对象都将触发`NoMethodError`异常

虽然`Integer()`可转换更多参数类型，但是对待字符串参数，它却比`#to_i`还挑剔

In [26]:
"ham sandwich".to_i

0

无论输入的原有类型是什么，你都渴望将其转换成特定原生类型。例如，无论输入的是`Float`类型的数字还是`nil`，甚至是十六进制字符串，只要可行，就将其转换成`Integer`类型

Ruby的类型转换方法可不止前面章节谈到的那些`#to_*`方法，Ruby核心模块还提供了一系列命名独特的强制类型转换方法，如`Array(), Float(), String(), Integer(),Rational()和Complex()`。通常，相对于对应`#to_*`版本的转换方法，它们可以转换更多的输入类型。

部分标准库也提供了强制类型转换方法

In [27]:
require 'pathname'
require 'uri'

path = Pathname.new("/etc/hosts")
p Pathname(path)
p Pathname("/etc/hosts")

uri_str = "http://example.org"
uri = URI.parse(uri_str)
p URI(uri_str)
p URI(uri)

#<Pathname:/etc/hosts>
#<Pathname:/etc/hosts>
#<URI::HTTP http://example.org>
#<URI::HTTP http://example.org>


#<URI::HTTP http://example.org>

下面的方法接受文件名为参数，报告文件大小。这里的文件名可以是`Pathname`对象，也可以是其他任何可悲转换为`Pathname`的对象

In [28]:
def file_size(filename)
  filename = Pathname(filename)
  filename.size
end

:file_size

In [29]:
file_size(Pathname.new("/etc/hosts"))
file_size("/etc/hosts")

1223

`Kernel#Array`会不遗余力地将输入转换为数组

In [30]:
Array("foo")

["foo"]

In [31]:
Array([1,2,3])

[1, 2, 3]

In [32]:
Array([])

[]

In [33]:
Array(nil)

[]

In [34]:
Array({:a =>  1, :b => 2})

[[:a, 1], [:b, 2]]

In [35]:
Array(1..5)

[1, 2, 3, 4, 5]

In [36]:
def log_reading(reading_or_readings)
  readings = Array(reading_or_readings)
  readings.each do |reading|
    puts "[READING] %3.2f" % reading.to_f
  end
end

:log_reading

In [37]:
log_reading(3.14)

[READING] 3.14


[3.14]

In [38]:
log_reading([])

[]

In [39]:
log_reading([87.5, 45.8765, 22])

[READING] 87.50
[READING] 45.88
[READING] 22.00


[87.5, 45.8765, 22]

In [40]:
log_reading(nil)

[]

一个对象提出的要求越少，就越容易被使用。若一个对象能为多种不同类型的的对象提供服务，那么将它作为一个辅助对象，在履行自身职责的时候，就不会向周围对象提出要求。

我们将采用Ruby核心类和标准库中强制类型转换方法的风格来定义一个名称特别的转换方法，它们目标类Point来命名

In [41]:
module Graphics
  module Conversions
    module_function
    def Point(*args)
      case args.first
      when Point then args.first
      when Array then Point.new(*args.first)
      when Integer then Point.new(*args)
      when String then
        Point.new(*args.first.split(':').map(&:to_i))
      else
        raise TypeError, "Cannot convert #{args.inspect} to Point"
      end
      end
      end
        Point = Struct.new(:x, :y) do
          def inspect
            "#{x}:#{y}"
          end
        end
      end

Graphics::Point

In [42]:
include Graphics
include Graphics::Conversions
Point(Point.new(2,3))
Point([9,7])
Point(3,5)
Point("8:10")

TypeError: Cannot convert [#<Point:0x00000055982d8c30 @x=2, @y=3, @name=nil>] to Point

接着我们就可以在代码中使用该转换方法了，尤其是和外来输入打交道的时候。无论何处，当不能确定输入类型时候，我们要做的不是浪费时间来摸清各种可能性，而是简单地用`Point()`方法将其包起来。然后，既然知道只会和`Point`对象打交道，便可将注意力转移到手边的业务逻辑上来。

`module_function`做了两件事，首先，将紧跟其后的方法都标注成了私有方法；其次，让紧跟其后的方法成了低昂前模块的单例方法

通过将#Point标注成私有方法，我们就可以将其封装在内部，从而有别于那些共有的接口方法。举个例子，假设有一个`canvas`对象已经引入了`Conversations`模块，那么应该没必要在该对象之外再来调用`canvas.Point(1,2)`吧。

In [43]:
def Point(*args)
  case args.first
    when Integer then Point.new(*args)
    when String then Point.new(*(args.first.split(':').map(&:to_i))
    when ->(arg) {arg.respond_to?(:to_point))}
      args.first.to_point
    when ->(arg) {arg.respond_to?(:to_ary)}
      Point.new(*args.first.to_ary)
    else
      raise TypeError, "Cannot convert #{args.inspect} to Point"
    end
    end

SyntaxError: <main>:4: syntax error, unexpected keyword_when, expecting ')'
    when ->(arg) {arg.respond_to?(:to...
    ^~~~
<main>:4: syntax error, unexpected ')', expecting '}'
...g) {arg.respond_to?(:to_point))}
...                              ^
<main>:6: syntax error, unexpected keyword_when, expecting '}'
    when ->(arg) {arg.respond_to?(:to...
    ^~~~
<main>:8: syntax error, unexpected keyword_else, expecting '}'
    else
    ^~~~
<main>:10: syntax error, unexpected keyword_end, expecting '}'
    end
    ^~~

  1. 标准库`#to_ary`，匹配数组，以及不是数组但可以转换为数组的对象
  2. 我们库中的`#to_point`，匹配那些定义了将自己转换为Point的对象

现在我们为程序提供了两个扩展点。我们不再排斥类数组对象（就差从`Array`继承了），并且现在的客户端代码可以自定义`#to_point`方法了（而非原来那样强制输入类型必须为`Point`）。

In [44]:
even = ->(x) {(x % 2) == 0}

#<Proc:0x00000055981705c8@<main>:0 (lambda)>

In [45]:
even === 4

true

In [46]:
even === 9

false

`case`语句使用三等运算符`（#===）` 来判定分支是否匹配，而`Ruby`的`Proc`对象正好定义了三等运算符（`#call`的别名）

In [47]:
class TrafficLight
  def change_to(state)
    @state = state
  end
  
  def signal
    case @state
    when "stop" then turn_on_lamp(:red)
    when "caution"
      turn_on_lamp(:yellow)
      rignt_warning_bell
    when "proceed" then turn_on_lamp(:green)
    end
    end
  
  def next_state
    case @state
    when "stop" then "proceed"
    when "caution" then "stop"
    when "proceed" then "caution"
    end
    end
      
  def turn_on_lamp(color)
    puts "Turning on #{color} lamp"
  end
      
  def ring_warning_bell
    puts "Ring ring ring"
  end
end

:ring_warning_bell

In [48]:
light = TrafficLight.new
light.change_to("PROCEED")
light.signal
puts "Next state:#{light.next_state.inspect}"

light.change_to(:stop)
light.signal
puts "Next state:#{light.next_state.inspect}"

Next state:nil
Next state:nil


该`case`并无`else`分支，这就意味着如果`@state`不是`stop`、`caution`或者`proceed`其中之一时，该方法会悄然出错。

我们可以通过改进`#change_to`方法，让它防范非法输入。

In [49]:
def change_to(state)
  raise ArgementError unless ["stop", "proceed", "caution"].include?(state)
  @state = state
end

:change_to

如果分别用独立的对象来表示红绿灯状态，会不会好点呢

In [56]:
class TrafficLight
  State = Struct.new(:name) do
    def to_s
      name
    end
  end
  
  VALID_STATES = [
    STOP = State.new("stop"),
    CAUTION = State.new("caution"),
    PROCEED = State.new("proceed")
    ]
  
  def change_to(state)
    raise ArgumentError unless VALID_STATES.include?(state)
    @state = state
    end
    
  def signal
    case @state
    when STOP then turn_on_lamp(:red)
    when CAUTION
      turn_on_lamp(:yellow)
      ring_warning_bell
    when PROCEED then turn_on_lamp(:green)
    end
    end
  
  def next_state
    case @state
    when STOP then 'proceed'
    when CAUTION then 'stop'
    when PROCEED then 'caution'
    end
    end    
    end

:next_state

我们再次探讨`state`的定义。这次将其申明为子类，而不再是实例对象。

In [65]:
class TrafficLight
  class State
    def to_s
      name
    end
    
    def name
      self.class.name.split('::').last.downcase
    end
    
    def signal(traffic_light)
      traffic_light.turn_on_lamp(color.to_sym)
    end
    
  end
    class Stop < State
      def color; 'red'; end
      
      def next_state; Proceed.new; end
    end
    
    class Caution < State
      def color; 'yellow'; end
      def next_state;  Stop.new; end
    
    
    
    def signal(traffic_light)
      super
      traffic_light.ring_warning_bell
    end
  end
  
  class Proceed < State
    def color; 'green'; end
    
    def next_state; Caution.new; end
  end
end

:next_state

In [61]:
class TrafficLight
  
  def next_state
    @state.next_state
  end
  
  def signal
    @state.signal(self)
  end
  
end

:signal

不幸的是,`TrafficLight`使用`State`的便利性反而变更糟了

In [66]:
light = TrafficLight.new
light.change_to(TrafficLight::Caution.new)
light.signal

ArgumentError: ArgumentError

In [67]:
class TrafficLight
  def change_to(state)
    @state = State(state)
  end
  
  private
  
  def State(state)
    case state
    when State then state
    else self.class.const_get(state.to_s.capitalize).new
    end
    end
  end

:State

现在可以这样使用了

In [68]:
light = TrafficLight.new
light.change_to(:caution)
light.signal
puts "Next state is: #{light.next_state}"

Turning on yellow lamp
Ring ring ring
Next state is: stop
